# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
X = df['message']
y = df[[col for col in df.columns if 'category' in col]]

In [7]:
print(X.shape, y.shape)
df.head()

(26216,) (26216, 36)


,id,message,original,genre,category_related,category_request,category_offer,category_aid_related,category_medical_help,category_medical_products,...,category_aid_centers,category_other_infrastructure,category_weather_related,category_floods,category_storm,category_fire,category_earthquake,category_cold,category_other_weather,category_direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
clf = RandomForestClassifier()
model = MultiOutputClassifier(estimator=clf)

In [6]:
clf = RandomForestClassifier()
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moc', MultiOutputClassifier(estimator=clf))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

**Before doing anything:** Notice that there are '2' in y, which does not make sense. I make the assumption to take the mode values, meaning that I replace all 2 by 0. 

In [ ]:
# replace all 2 in y by 0 
for col in y.columns:
    print(col, y[col].unique())
y = y.replace(2, 0)
print('After replacing, we have that the suspecious category now contains only unique values ', y['category_related'].unique())

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [23]:
print(X_train.shape, X_test.shape, y_pred.shape)
np.transpose(y_pred).shape
df.columns

(19662,) (6554,) (6554, 36)


Index(['id', 'message', 'original', 'genre', 'category_related',
       'category_request', 'category_offer', 'category_aid_related',
       'category_medical_help', 'category_medical_products',
       'category_search_and_rescue', 'category_security', 'category_military',
       'category_child_alone', 'category_water', 'category_food',
       'category_shelter', 'category_clothing', 'category_money',
       'category_missing_people', 'category_refugees', 'category_death',
       'category_other_aid', 'category_infrastructure_related',
       'category_transport', 'category_buildings', 'category_electricity',
       'category_tools', 'category_hospitals', 'category_shops',
       'category_aid_centers', 'category_other_infrastructure',
       'category_weather_related', 'category_floods', 'category_storm',
       'category_fire', 'category_earthquake', 'category_cold',
       'category_other_weather', 'category_direct_report'],
      dtype='object')

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [26]:
for i in np.arange(y_pred.shape[1]):
    pred = np.transpose(y_pred)[i]
    true = y_test[df.columns[i+4]] # select the corrresponding column from y_test
    print(df.columns[i+4],'\n', classification_report(true, pred)) # print the report on each category

category_related 
              precision    recall  f1-score   support

          0       0.63      0.37      0.46      1512
          1       0.82      0.93      0.88      5000
          2       0.17      0.02      0.04        42

avg / total       0.77      0.80      0.78      6554

category_request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5462
          1       0.80      0.38      0.51      1092

avg / total       0.87      0.88      0.86      6554

category_offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6526
          1       0.00      0.00      0.00        28

avg / total       0.99      1.00      0.99      6554

category_aid_related 
              precision    recall  f1-score   support

          0       0.73      0.87      0.79      3840
          1       0.75      0.53      0.62      2714

avg / total       0.74      0.73      0.72      6554

category_

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.